In [3]:
import pyodbc
import json
import datetime

from kafka import KafkaProducer
from kafka.vendor.six import b

import argparse
from uuid import uuid4

from six.moves import input

from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import StringSerializer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer


schema_str = """
{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "title": "User",
  "properties": {
    "amount": {
      "type": "number"
    },
    "currency": {
      "type": "string"
    },
    "ext_acc": {
      "type": "integer"
    },
    "file_path": {
      "type": "string"
    },
    "modified_by": {
      "type": "string"
    }
  },
  "required": [
    "ext_acc",
    "modified_by",
    "currency",
    "amount",
    "file_path"
  ],
  "type": "object"
}
"""




class User(object):
    """
    User record
    Args:
        name (str): User's name
        favorite_number (int): User's favorite number
        favorite_color (str): User's favorite color
        address(str): User's address; confidential
    """
    def __init__(self, ext_acc, modified_by, currency, amount, file_path):
        self.ext_acc = ext_acc
        self.modified_by = modified_by
        self.currency = currency
        self.amount = amount
        self.file_path = file_path

        
def user_to_dict(user, ctx):
    """
    Returns a dict representation of a User instance for serialization.
    Args:
        user (User): User instance.
        ctx (SerializationContext): Metadata pertaining to the serialization
            operation.
    Returns:
        dict: Dict populated with user attributes to be serialized.
    """
    return dict(ext_acc=user.ext_acc,
                modified_by=user.modified_by,
                currency=user.currency,
                amount=user.amount,
                file_path=user.file_path)


def delivery_report(err, msg):
    """
    Reports the failure or success of a message delivery.
    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.
    Note:
        In the delivery report callback the Message.key() and Message.value()
        will be the binary format as encoded by any configured Serializers and
        not the same object that was passed to produce().
        If you wish to pass the original object(s) for key and value to delivery
        report callback we recommend a bound callback or lambda where you pass
        the objects along.
    """
    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))



schema_registry_conf = {'url':'http://localhost:8081' }
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

json_serializer = JSONSerializer(schema_str, schema_registry_client, user_to_dict)

producer_conf = {'bootstrap.servers': 'localhost:9092',
                         'key.serializer': StringSerializer('utf_8'),
                         'value.serializer': json_serializer}

producer = SerializingProducer(producer_conf)

server = 'localhost' 
database = 'TestDB' 
username = 'sa' 
password = 'Password1#' 
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

rows = cursor.execute('SELECT ext_acc,modified_by,currency,amount,file_path FROM citadel ')

for row in rows :

    items = [dict(zip([key[0] for key in cursor.description], row))  for row in rows ]

    class DateTimeEncoder(json.JSONEncoder):
        def default(self, z):
            if isinstance(z, datetime.datetime):
                return (str(z))
            else:
                return super().default(z)

    for item in items:
        
        item_key=str(item["ext_acc"]) 
    
        user = User(ext_acc=item['ext_acc'],
                modified_by=item['modified_by'],
                currency=item['currency'],
                amount=item['amount'],
                file_path=item['file_path'])
        producer.produce('excesscash_topic_new', key=str(uuid4()), value=user, on_delivery=delivery_report)
        print (item_key, item)

print("\nStarted Flushing records..............\n")
producer.flush()
print("\nCompelted Flushing records..............\n")


1111 {'ext_acc': 1111, 'modified_by': 'xceptor', 'currency': 'USD    ', 'amount': 100.0, 'file_path': 'FILE2               '}
2222 {'ext_acc': 2222, 'modified_by': 'xceptor', 'currency': 'USD    ', 'amount': 200.0, 'file_path': 'FILE3               '}
3333 {'ext_acc': 3333, 'modified_by': 'xceptor', 'currency': 'USD    ', 'amount': 300.0, 'file_path': 'FILE4               '}
4444 {'ext_acc': 4444, 'modified_by': 'xceptor', 'currency': 'USD    ', 'amount': 400.0, 'file_path': 'FILE5               '}
5555 {'ext_acc': 5555, 'modified_by': 'xceptor', 'currency': 'EUR    ', 'amount': 55356.0, 'file_path': 'FILEXXXX            '}
6666 {'ext_acc': 6666, 'modified_by': 'xdo    ', 'currency': 'USD    ', 'amount': 600.0, 'file_path': 'FILE7               '}
7777 {'ext_acc': 7777, 'modified_by': 'xdo    ', 'currency': 'USD    ', 'amount': 700.0, 'file_path': 'FILE8               '}
8888 {'ext_acc': 8888, 'modified_by': 'xdo    ', 'currency': 'USD    ', 'amount': 800.0, 'file_path': 'FILE9        